# Model soup

In [1]:
# 기본코드
import inspect
import os
import sys
import time
import numpy as np

import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint, load_state_dict
from mmcv.parallel import MMDataParallel

def uniform_soup(cfg, model, checkpoint_paths ,device = "cpu", by_name = False):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model
    
    dataset = build_dataset(cfg.data.val)
    data_loader = build_dataloader(
            dataset,
            samples_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)  
    
    
    model = model.to(device)
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        weight_dict = checkpoint['state_dict']
        for k, v in weight_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    
    load_state_dict(model, model_dict)
    model.CLASSES = dataset.CLASSES
    model = MMDataParallel(model.cuda(), device_ids=[0])
    output = single_gpu_test(model, data_loader)
    eval_kwargs = {}
    eval_kwargs.update(metric=['mIoU'])
    metric = dataset.evaluate(output, **eval_kwargs)
    print(f"mIoU: {metric['mIoU']}")
    
    return model, checkpoint

def greedy_soup(cfg, model_ori, checkpoint_paths, device = "cpu"):
    try:
        import torch
    except:
        print("If you want to use 'Model Soup for Torch', please install 'torch'")
        return model_ori
    
    dataset = build_dataset(cfg.data.val)
    data_loader = build_dataloader(
            dataset,
            samples_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)  
    
    
    result = []
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        model = model_ori.to(device)
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        model.CLASSES = dataset.CLASSES
        model = MMDataParallel(model.cuda(), device_ids=[0])
        output = single_gpu_test(model, data_loader)
        eval_kwargs = {}
        eval_kwargs.update(metric=['mIoU'])
        metric = dataset.evaluate(output, **eval_kwargs)
        result.append((metric['mIoU'],checkpoint_path))
        print(f"리스트에 {i}번째 mIoU {metric['mIoU']}저장")
    
    result.sort(key = lambda x : x[0], reverse = True)
    print(f"리스트 정렬")
    print(result)
    
    model = model_ori.to(device)
    model_dict = model.state_dict()
    pre_metric_value = 0
    pre_weight_dict = {}
    for i, (mIoU, checkpoint_path) in enumerate(result):
        model = model_ori.to(device)
        soups = {key:[] for key in model_dict}
        now_model_dict = model_dict
        if i == 0:
            checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
            pre_metric_value = mIoU
            pre_weight_dict = checkpoint['state_dict']
            print("soup 모델에 가장 높은 mIou를 가진 checkpoint가 추가되었습니다")
            print(f"추가된 checkpoint_path: {checkpoint_path}")
            print(f"현재 최고 mIoU: {pre_metric_value}")
        else:
            checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
            weight_dict = checkpoint['state_dict']
            
            for k, v in pre_weight_dict.items():
                soups[k].append(v)
            for k, v in weight_dict.items():
                soups[k].append(v)    
            if 0 < len(soups):
                soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
                now_model_dict.update(soups)
                
                
            load_state_dict(model, now_model_dict)
            model.CLASSES = dataset.CLASSES
            model = MMDataParallel(model.cuda(), device_ids=[0])
            output = single_gpu_test(model, data_loader)
            eval_kwargs = {}
            eval_kwargs.update(metric=['mIoU'])
            metric = dataset.evaluate(output, **eval_kwargs)
            
            if metric['mIoU'] >= pre_metric_value:
                pre_metric_value = metric['mIoU']
                pre_weight_dict = now_model_dict
                print("soup 모델에 새로운 checkpoint가 추가되었습니다")
                print(f"추가된 checkpoint_path: {checkpoint_path}")
                print(f"현재 최고 mIoU: {pre_metric_value}")
            else:
                print("이번 체크 포인트는 soup 모델에 추가되지 않았습니다")
                print(f"이번 checkpoint_path: {checkpoint_path}")
                print(f"현재 최고 mIoU: {pre_metric_value}, 이번 mIou {metric['mIoU']}")
            
    model = model_ori.to(device)
    load_state_dict(model, pre_weight_dict)
    return model, checkpoint

/opt/conda/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0. 모델 & checkpoint 가져오기

In [24]:
################ model cfg path 적기 ################
cfg = Config.fromfile('/opt/ml/input/code/mmsegmentation/configs/segformer/segformer_mit-b4.py')
################ model cfg path 적기 ################
model = build_segmentor(cfg.model)

################ soup할 checkpoint path 적기 ################
checkpoint_paths = [
    '/opt/ml/input/code/mmsegmentation/work_dir/seg-b4/best_mIoU_epoch_63.pth',
    '/opt/ml/input/code/mmsegmentation/work_dir/seg-combo/best_mIoU_epoch_86.pth',
    '/opt/ml/input/code/mmsegmentation/work_dir/seg-combo/epoch_100.pth',
    '/opt/ml/input/code/mmsegmentation/work_dir/pseudo_seg/best_mIoU_epoch_92.pth',
    '/opt/ml/input/code/mmsegmentation/work_dir/pseudo_seg/epoch_100.pth',
]
################ soup할 checkpoint path 적기 ################
device = "cpu"

/opt/conda/lib/python3.8/site-packages/mmsegmentation-0.29.1-py3.8.egg/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


1. uniform soup

In [25]:
################ save dir path 적기 ################
save_dir_path = './'
name = 'mysoup' # soup 이름 적기
################ save dir path 적기 ################

print("\n[Uniform Soup]")
uniform_model, checkpoint = uniform_soup(cfg, model, checkpoint_paths, device = device)
uniform_dict = checkpoint
uniform_dict['state_dict'] = uniform_model.state_dict()

torch.save(uniform_dict, save_dir_path+f'uniform_model_soup_{name}.pth')

2023-01-05 06:44:08,982 - mmseg - INFO - Loaded 655 images



[Uniform Soup]
load checkpoint from local path: /opt/ml/input/code/mmsegmentation/work_dir/pseudo_seg/best_mIoU_epoch_92.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 7.0 task/s, elapsed: 94s, ETA:     0sper class results:

+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|   Backgroud   | 96.65 | 98.54 |
| General trash | 49.17 | 59.11 |
|     Paper     | 80.51 | 91.77 |
|   Paper pack  |  56.9 | 67.43 |
|     Metal     |  68.9 | 77.73 |
|     Glass     | 66.71 | 80.97 |
|    Plastic    | 52.73 | 69.28 |
|   Styrofoam   | 83.01 | 86.81 |
|  Plastic bag  | 86.21 | 92.11 |
|    Battery    | 77.04 | 92.42 |
|    Clothing   | 58.46 | 72.02 |
+---------------+-------+-------+
Summary:

+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 94.69 | 70.57 | 80.74 |
+-------+-------+-------+
mIoU: 0.7056999999999999


2. Greedy Soup (uniform weight update)

In [19]:
################ save dir path 적기 ################
save_dir_path = './'
name = 'mysoup' # soup 이름 적기
################ save dir path 적기 ################

print("[Greedy Soup (uniform weight update)]")
greedy_model, checkpoint = greedy_soup(cfg, model, checkpoint_paths, device = device)
greedy_dict = checkpoint
greedy_dict['state_dict'] = greedy_model.state_dict()
torch.save(greedy_dict, save_dir_path+f'greedy_model_soup_{name}.pth')


2023-01-05 02:00:21,059 - mmseg - INFO - Loaded 655 images


[Greedy Soup (uniform weight update)]
load checkpoint from local path: /opt/ml/input/code/mmsegmentation/work_dir/seg-b4/best_mIoU_epoch_63.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 6.9 task/s, elapsed: 95s, ETA:     0sper class results:

+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|   Backgroud   | 96.45 | 98.84 |
| General trash | 53.13 | 61.88 |
|     Paper     | 79.99 | 88.88 |
|   Paper pack  | 56.94 | 77.07 |
|     Metal     | 64.19 | 70.79 |
|     Glass     | 66.54 | 77.39 |
|    Plastic    | 54.68 | 71.26 |
|   Styrofoam   | 74.14 | 81.54 |
|  Plastic bag  | 86.53 | 92.08 |
|    Battery    | 74.28 | 98.55 |
|    Clothing   | 61.56 | 69.68 |
+---------------+-------+-------+
Summary:

+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 94.64 | 69.86 | 80.73 |
+-------+-------+-------+
리스트에 0번째 mIoU 0.6986저장
load checkpoint from local path: /opt/ml/input/code/mmsegmentation/work_dir/seg-comb